In [2]:
from PIL import Image
from glob import glob
from tqdm import tqdm
import os
import io
import time
import json
import base64
import urllib
import pickle
import requests

def translation(input_str:str):
    client_id = "YOUR ID"
    client_secret = "YOUR SECRET"
    url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"

    data = input_str
    encText = urllib.parse.quote(data)
    data = "source=ko&target=en&text=" + encText # source = 원본 언어, target = 변환할 언어

    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        en_text = json.loads(response_body.decode('utf-8'))['message']['result']['translatedText']
        return en_text
    
# text2img
def t2i(prompt, negative_prompt):
    r = requests.post(
        'https://api.kakaobrain.com/v2/inference/karlo/t2i',
        json = {
            "version": "v2.1", 
            "prompt": prompt,
            "negative_prompt": negative_prompt, 
            "height": 1024,
            "width": 1024,
            "return_type": "base64_string",
            "prior_num_inference_steps": 25,
            "prior_guidance_scale": 5.0,
            "num_inference_steps": 50 ,
            "guidance_scale": 5.0,
            "samples": "8"
        },
        headers = {
            'Authorization': f'KakaoAK YOUR KEY',
            'Content-Type': 'application/json'
        }
    )
    # 응답 JSON 형식으로 변환
    response = json.loads(r.content)
    return response

def make_infer_target_dict(control_dict: dict, *args):
    # make example_dict
    example_dict = {
        translation(user_input):user_input for user_input in control_dict["user_input_lst"]
    }

    
    final_dict = {}
    """
    make data_dict
    tmp_lst = [(prompt, negtive_prompt, org_example), ...]
    """
    
    # 시험하고 싶은 옵션 각 1개씩 추출
    for ele in args:
        for key, value in control_dict[ele].items():
            tmp_lst = [
                (
                    ", ".join(
                        [
                            example, 
                            key, 
                            value["pos"], 
                            control_dict["fixed_prompt_default"]
                        ]
                    ), 
                    ", ".join(
                        [
                            control_dict["fixed_negative_prompt_default"], 
                            value["neg"]
                        ]
                    ), 
                    org_example
                ) for example, org_example in example_dict.items()
            ]
            final_dict[key] = tmp_lst

    # 시험하고 싶은 옵션이 2개인 경우 동시 적용
    for first_ele in control_dict[args[0]]:
        for second_ele in control_dict[args[1]]:
            key = f"{first_ele}_{second_ele}"
            tmp_lst = [
                (
                    ", ".join(
                        [
                            example,
                            first_ele, 
                            second_ele, 
                            control_dict[args[0]][first_ele]["pos"], 
                            control_dict[args[1]][second_ele]["pos"], 
                            control_dict["fixed_prompt_default"]
                        ]
                    ).replace(", ,",",").strip(),
                    ", ".join(
                        [
                            control_dict["fixed_negative_prompt_default"], 
                            control_dict[args[0]][first_ele]["neg"], 
                            control_dict[args[1]][second_ele]["neg"]
                        ]
                    ).replace(", ,",",").strip(),
                    org_example
                ) for example, org_example in example_dict.items()
            ]
            final_dict[key] = tmp_lst

    return final_dict

def infer(data_dict:dict, root:str):
    for option, example_lst in data_dict.items():
        os.makedirs(root + "/" + option, exist_ok = True)
        
        for example in tqdm(example_lst):
            prompt = example[0]
            negative_prompt = example[1]
            user_input = example[2]

            response = t2i(prompt, negative_prompt)

            save_folder_path = root + f"/{option}/"

            for img_num in range(8):
                img_str = response["images"][img_num]["image"]
                imgdata = base64.b64decode(img_str)
                dataBytesIO = io.BytesIO(imgdata)
                PIL_img = Image.open(dataBytesIO)

                PIL_img.save(save_folder_path + user_input + f",{option}" + f"_{str(img_num).zfill(4)}" + ".jpg")
                    
            
control_dict = {
    "fixed_prompt_default": "high quality, hyper-detailed, best composition, simple drawing, flat, for storyboard cut scene",
    "fixed_negative_prompt_default": "out of frame, low resolution, blurry, worst quality, fuzzy, text, low quality, normal quality, signature, watermark, grainy, bad proportions, cropped, duplicate, malformed, cropped, pixelated, poorly drawn hands, poorly drawn feet, poorly drawn face, body out of frame, distorted face, bad anatomy, missing anatomy, missing body, missing face, missing legs, missing fingers, missing feet, missing toe, fewer digits, extra limbs, extra anatomy, extra face, extra arms, extra fingers, extra hands, extra legs, extra feet, extra toe, mutated hands, ugly, mutilated, disfigured, mutation, bad proportions, cropped head, cross-eye, mutilated, distorted eyes, detailed face, distorted hands, strabismus",
    
    "style": {
        " ": {
            "pos": "",
            "neg": ""
        },
        "Pencil sketch with no color": {
            "pos": "",
            "neg": "Real pencil"
        }, 
        "Colored pencil": {
            "pos": "",
            "neg": "Real pencil"
        }, 
        "Webtoon": {
            "pos": "Similar picture",
            "neg": ""
        }, 
        "Watercolor": {
            "pos": "",
            "neg": ""
        }
    },
    
    "angle": {
        " ": {
            "pos": "",
            "neg": ""
        },
        "High angle shot": {
            "pos": "",
            "neg": ""
        }, 
        "Low angle shot": {
            "pos": "",
            "neg": ""
        }, 
        "Eye level shot": {
            "pos": "",
            "neg": ""
        }, 
        "Close up angle": {
            "pos": "",
            "neg": ""
        }, 
        "Full shot": {
            "pos": "",
            "neg": ""
        }, 
        "Overhead view": {
            "pos": "",
            "neg": ""
        },
    },
    
    "user_input_lst": [
        "고양이,나른한 오후,낮잠", 
        "웃고있는 남자,손에 음료수,청바지",
        "초원위의 집,나무 한그루"
    ]
}

data_dict = make_infer_target_dict(control_dict, "style", "angle")

pickle_name = "data_dict.pkl"
with open(pickle_name, "wb")as f:
    pickle.dump(data_dict, f)

/home/ubuntu/anaconda3/envs/3D/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
with open(pickle_name, "rb")as f:
    data = pickle.load(f)
    
infer(
    data_dict = data,
    root = "ver1",
)

100%|██████████| 3/3 [01:04<00:00, 21.46s/it]


In [12]:
data

{'Pencil sketch with no color_High angle shot': [('a cat, a drowsy afternoon, a nap, Pencil sketch with no color, High angle shot, , high quality, hyper-detailed, best composition, simple facial expression, simple drawing, flat, for storyboard cut scene',
   'out of frame, low resolution, blurry, worst quality, fuzzy, text, low quality, normal quality, signature, watermark, grainy, bad proportions, cropped, duplicate, malformed, cropped, pixelated, poorly drawn hands, poorly drawn feet, poorly drawn face, body out of frame, distorted face, bad anatomy, missing anatomy, missing body, missing face, missing legs, missing fingers, missing feet, missing toe, fewer digits, extra limbs, extra anatomy, extra face, extra arms, extra fingers, extra hands, extra legs, extra feet, extra toe, mutated hands, ugly, mutilated, disfigured, mutation, bad proportions, cropped head, cross-eye, mutilated, distorted eyes, detailed face, distorted hands, strabismus, Real pencil,',
   '고양이,나른한 오후,낮잠'),
  ('A 